In [1]:
#! /usr/bin/env python

from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

In [2]:
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

n = 5
raw_data = sc.textFile("/data/twitter/twitter_sample_small.txt")
edges = raw_data.map(parse_edge).cache()

#edges from follower nodes to main nodes
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

def step(item):
    # prev_v_l : previous vertices list
    # prev_l : previous vertex
    prev_v, prev_v_l, next_v = item[0], item[1][0], item[1][1]
    return (next_v, prev_v_l + [next_v])

x = 12
prev_v = [12]
paths = sc.parallelize([(x, prev_v)]).partitionBy(n)
flag = 0
while True:
    paths = paths.join(forward_edges, n).map(step)
    flag = paths.filter(lambda i: i[0] == 34).count()
    if flag:
        break

paths = paths.filter(lambda i: i[0] == 34).collect()
prev_v_l = paths[0][1]
print(','.join(map(str, prev_v_l)))

12,422,53,52,107,20,23,274,34
